In [ ]:
# set parameters
ADAPTER_OR_MODEL_PATH = r"E:\FineTuning\models\qwen25coder3b_codealpaca_lora\adapter"  # adapter dir OR merged model dir
BASE_MODEL = "Qwen/Qwen2.5-Coder-3B-Instruct"  # base model if USE_ADAPTER=False
USE_ADAPTER = True
USE_4BIT = True
MAX_NEW_TOKENS = 216
TIMEOUT_SEC = 12
PRINT_EVERY = 10

In [2]:
import os, json, tempfile, subprocess, sys, time
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

c:\Users\yijin\anaconda3\envs\tf-gpu\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\yijin\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PROMPT_SYS = "You are a careful Python coding assistant. Write only valid Python code for the function, no explanations."
GEN_INSTR = (
    "You are given a Python function signature and docstring.\n"
    "Write the COMPLETE function, starting with the `def ...:` line.\n"
    "Return ONLY the code (no comments, no backticks, no extra text).\n"
    "Function to implement:\n{prompt}\n"
)

In [4]:
def load_model_and_tok(adapter_or_model_path, base_model=None, use_adapter=True, use_4bit=True):
    dtype = torch.bfloat16
    if use_adapter:
        if use_4bit:
            from transformers import BitsAndBytesConfig
            bnb = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=dtype,
            )
            model = AutoModelForCausalLM.from_pretrained(
                base_model, device_map="auto", trust_remote_code=True,
                quantization_config=bnb, torch_dtype=dtype,
                attn_implementation="sdpa",
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                base_model, device_map="auto", trust_remote_code=True, torch_dtype=dtype
            )
        tok = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
        if tok.pad_token is None: tok.pad_token = tok.eos_token
        tok.padding_side = "left"
        model = PeftModel.from_pretrained(model, adapter_or_model_path)
        model.eval()
        return model, tok
    else:
        tok = AutoTokenizer.from_pretrained(adapter_or_model_path, trust_remote_code=True)
        if tok.pad_token is None: tok.pad_token = tok.eos_token
        tok.padding_side = "left"
        model = AutoModelForCausalLM.from_pretrained(
            base_model, device_map="auto", trust_remote_code=True,
            torch_dtype=dtype
        )
        model.eval()
        return model, tok

In [5]:
def generate_solution(model, tok, prompt, max_new_tokens=216):
    messages = [
        {"role": "system", "content": PROMPT_SYS},
        {"role": "user", "content": GEN_INSTR.format(prompt=prompt)},
    ]
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,                # deterministic for eval
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.eos_token_id,
        )

    # decode ONLY newly generated tokens (skip the prompt part)
    gen_tokens = out[0][inputs["input_ids"].shape[-1]:]
    gen = tok.decode(gen_tokens, skip_special_tokens=True)
    gen = gen.replace("```python", "").replace("```", "").strip()
    return gen

In [6]:
def run_tests(solution_code: str, test_code: str, timeout_sec=10):
    with tempfile.TemporaryDirectory() as td:
        pth = os.path.join(td, "sol.py")
        with open(pth, "w", encoding="utf-8") as f:
            f.write(solution_code + "\n\n" + test_code + "\n")
        try:
            p = subprocess.run([sys.executable, pth],
                               stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                               timeout=timeout_sec, check=False)
            ok = (p.returncode == 0)
            return ok, p.stdout.decode(), p.stderr.decode()
        except subprocess.TimeoutExpired:
            return False, "", "Timeout"

In [7]:
def evaluate(model, tok):
    ds = load_dataset("openai_humaneval")["test"]
    print("Number of tasks:", len(ds))
    total, passed = 0, 0
    rows = []
    start = time.time()
    for i, ex in enumerate(ds):
        task_id = ex["task_id"]; prompt = ex["prompt"]; test_code = ex["test"]
        sol = generate_solution(model, tok, prompt, max_new_tokens=MAX_NEW_TOKENS)
        ok, out, err = run_tests(sol, test_code, timeout_sec=TIMEOUT_SEC)
        total += 1; passed += int(ok)
        if (i+1) % PRINT_EVERY == 0 or i == 0:
            print(f"[{i+1}/{len(ds)}] {task_id} -> PASS={ok}  (running {passed}/{total} = {passed/total:.3f})")
        rows.append({"task_id": task_id, "pass": bool(ok)})
    elapsed = time.time() - start
    print(f"\nScore: {passed}/{total} = {passed/total:.3f}   (time: {elapsed/60:.1f} min)")
    return rows

In [8]:
# fine tuned model
model, tok = load_model_and_tok(
    ADAPTER_OR_MODEL_PATH, base_model=BASE_MODEL,
    use_adapter=True, use_4bit=USE_4BIT
)
res_original = evaluate(model, tok)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.82s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Number of tasks: 164


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[1/164] HumanEval/0 -> PASS=False  (running 0/1 = 0.000)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

...

[150/164] HumanEval/149 -> PASS=True  (running 89/150 = 0.593)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

[160/164] HumanEval/159 -> PASS=False  (running 95/160 = 0.594)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Score: 99/164 = 0.604   (time: 13.0 min)
